# Credit Card Fraud Detection - Data Cleaning & EDA
## Professional Data Cleaning Pipeline with ACTUAL Data Cleaning

**Author:** Data Science Team  
**Date:** February 2026  
**Dataset:** Credit Card Transactions (10,000 records)

---

## สิ่งที่โน้ตบุ๊คนี้ทำ:
1. ✅ **ตรวจสอบคุณภาพข้อมูล** (Data Quality Check)
2. ✅ **ทำความสะอาดข้อมูล** (Data Cleaning)
   - ลบ/แก้ไขค่าที่ไม่ถูกต้อง
   - จัดการค่า Outliers
   - จัดการค่า Missing (ถ้ามี)
3. ✅ **วิเคราะห์ข้อมูล** (EDA)
4. ✅ **บันทึกข้อมูลที่สะอาดแล้ว**

## 1. Setup and Imports

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, ks_2samp, mannwhitneyu
import warnings
import os
from io import StringIO

warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✓ Libraries imported successfully")
print("✓ Configuration set")

## 2. Load Data

In [ ]:
# Define file paths
input_filepath = 'Raw_Data/credit_card_fraud_10k.csv'
output_filepath = 'Cleaned_Data/credit_card_fraud_10k_cleaned.csv'

# Create output directory if it doesn't exist
output_dir = os.path.dirname(output_filepath)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"✓ Created output directory: {output_dir}")

# Load data
print(f"Loading data from: {input_filepath}")
df_raw = pd.read_csv(input_filepath)
print(f"✓ Loaded {len(df_raw)} records from file")
print("="*80)
print("FRAUD DETECTION DATA PIPELINE - INITIALIZED")
print("="*80)

# Create a copy for cleaning
df = df_raw.copy()
print(f"\n✓ Created working copy of data")

## 3. Data Quality Assessment (BEFORE Cleaning)
### Step 1: Schema Validation

In [ ]:
print("\n[STEP 1] SCHEMA VALIDATION")
print("-" * 80)

expected_schema = {
    'transaction_id': 'int64',
    'amount': 'float64',
    'transaction_hour': 'int64',
    'merchant_category': 'object',
    'foreign_transaction': 'int64',
    'location_mismatch': 'int64',
    'device_trust_score': 'int64',
    'velocity_last_24h': 'int64',
    'cardholder_age': 'int64',
    'is_fraud': 'int64'
}

schema_valid = True
for col, dtype in expected_schema.items():
    if col not in df.columns:
        print(f"  ✗ Missing column: {col}")
        schema_valid = False
    elif df[col].dtype != dtype:
        print(f"  ⚠ Column {col}: Expected {dtype}, got {df[col].dtype}")

if schema_valid:
    print("  ✓ Schema validation PASSED")

### Step 2: Missing Value Analysis

In [ ]:
print("\n[STEP 2] MISSING VALUE ANALYSIS")
print("-" * 80)

missing_summary = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum().values,
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).values,
    'Data_Type': df.dtypes.values
})

print(missing_summary.to_string(index=False))

total_missing = df.isnull().sum().sum()
if total_missing == 0:
    print("\n  ✓ No missing values detected")
else:
    print(f"\n  ⚠ Total missing values: {total_missing}")

### Step 3: Duplicate Detection

In [ ]:
print("\n[STEP 3] DUPLICATE DETECTION")
print("-" * 80)

# Check duplicate rows
duplicate_rows = df.duplicated()
dup_count = duplicate_rows.sum()

# Check duplicate transaction IDs
dup_ids = df['transaction_id'].duplicated()
dup_id_count = dup_ids.sum()

print(f"  Duplicate Rows: {dup_count} ({dup_count/len(df)*100:.2f}%)")
print(f"  Duplicate Transaction IDs: {dup_id_count}")

if dup_count > 0:
    print("\n  Duplicate Records:")
    print(df[duplicate_rows])
else:
    print("\n  ✓ No duplicates detected")

### Step 4: Domain Constraint Validation

In [ ]:
print("\n[STEP 4] DOMAIN CONSTRAINT VALIDATION (BEFORE CLEANING)")
print("-" * 80)

validation_results = {}

# Transaction Hour: 0-23
invalid_hours = df[(df['transaction_hour'] < 0) | (df['transaction_hour'] > 23)]
validation_results['transaction_hour'] = len(invalid_hours)
print(f"  transaction_hour (0-23): {len(invalid_hours)} invalid values")
if len(invalid_hours) > 0:
    print(f"    → Invalid values: {invalid_hours['transaction_hour'].unique()}")

# Device Trust Score: 0-100
invalid_scores = df[(df['device_trust_score'] < 0) | (df['device_trust_score'] > 100)]
validation_results['device_trust_score'] = len(invalid_scores)
print(f"  device_trust_score (0-100): {len(invalid_scores)} invalid values")
if len(invalid_scores) > 0:
    print(f"    → Invalid values: {invalid_scores['device_trust_score'].unique()}")

# Amount: positive
invalid_amounts = df[df['amount'] <= 0]
validation_results['amount'] = len(invalid_amounts)
print(f"  amount (>0): {len(invalid_amounts)} invalid values")
if len(invalid_amounts) > 0:
    print(f"    → Invalid transaction IDs: {invalid_amounts['transaction_id'].tolist()}")
    print(f"    → Invalid amounts: {invalid_amounts['amount'].tolist()}")

# Velocity: non-negative
invalid_velocity = df[df['velocity_last_24h'] < 0]
validation_results['velocity'] = len(invalid_velocity)
print(f"  velocity_last_24h (≥0): {len(invalid_velocity)} invalid values")
if len(invalid_velocity) > 0:
    print(f"    → Invalid values: {invalid_velocity['velocity_last_24h'].unique()}")

# Age: reasonable range
invalid_age = df[(df['cardholder_age'] < 18) | (df['cardholder_age'] > 100)]
validation_results['cardholder_age'] = len(invalid_age)
print(f"  cardholder_age (18-100): {len(invalid_age)} invalid values")
if len(invalid_age) > 0:
    print(f"    → Invalid values: {invalid_age['cardholder_age'].unique()}")

# Binary fields: 0 or 1
binary_fields = ['foreign_transaction', 'location_mismatch', 'is_fraud']
for field in binary_fields:
    invalid_binary = df[~df[field].isin([0, 1])]
    validation_results[field] = len(invalid_binary)
    print(f"  {field} (0/1): {len(invalid_binary)} invalid values")
    if len(invalid_binary) > 0:
        print(f"    → Invalid values: {invalid_binary[field].unique()}")

total_invalid = sum(validation_results.values())
if total_invalid == 0:
    print("\n  ✓ All domain constraints satisfied")
else:
    print(f"\n  ⚠ Total domain violations: {total_invalid}")
    print(f"  → These will be CLEANED in the next steps")

### Step 5: Outlier Detection

In [ ]:
print("\n[STEP 5] OUTLIER DETECTION (BEFORE CLEANING)")
print("-" * 80)

numerical_features = ['amount', 'transaction_hour', 'device_trust_score', 
                     'velocity_last_24h', 'cardholder_age']

outlier_summary = []

for feature in numerical_features:
    # IQR Method
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    iqr_outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]
    
    # Z-Score Method (threshold = 3)
    z_scores = np.abs(stats.zscore(df[feature]))
    z_outliers = df[z_scores > 3]
    
    outlier_summary.append({
        'Feature': feature,
        'IQR_Outliers': len(iqr_outliers),
        'Z_Score_Outliers': len(z_outliers),
        'Lower_Bound': lower_bound,
        'Upper_Bound': upper_bound,
        'Min': df[feature].min(),
        'Max': df[feature].max()
    })

outlier_df = pd.DataFrame(outlier_summary)
print(outlier_df.to_string(index=False))
print("\n  Note: Outliers in fraud detection are often IMPORTANT (fraud cases)")
print("        We will keep them unless they are clearly data errors")

## 4. 🧹 DATA CLEANING (ทำความสะอาดจริงๆ)
### เริ่มต้นการ Clean ข้อมูล

In [ ]:
print("\n" + "="*80)
print("🧹 DATA CLEANING PROCESS (การทำความสะอาดข้อมูลจริง)")
print("="*80)

rows_before = len(df)
cleaning_log = []

print(f"\nStarting with: {rows_before} records")
print("\nCleaning steps:")

### Clean 1: Remove/Fix Invalid Amounts

In [ ]:
print("\n[CLEAN 1] Handling Invalid Amounts (amount <= 0)")
print("-" * 80)

invalid_amounts = df[df['amount'] <= 0]
print(f"  Found {len(invalid_amounts)} records with invalid amounts")

if len(invalid_amounts) > 0:
    print(f"\n  Invalid records:")
    print(invalid_amounts[['transaction_id', 'amount', 'is_fraud']].to_string(index=False))
    
    # Decision: Remove these records (can't have transactions with zero/negative amounts)
    df = df[df['amount'] > 0]
    
    removed = len(invalid_amounts)
    print(f"\n  ✓ Removed {removed} records with invalid amounts")
    cleaning_log.append(f"Removed {removed} records: amount <= 0")
else:
    print("  ✓ No invalid amounts found")

print(f"\n  Records after cleaning: {len(df)} (removed: {rows_before - len(df)})")

### Clean 2: Remove Duplicates (if any)

In [ ]:
print("\n[CLEAN 2] Removing Duplicate Records")
print("-" * 80)

rows_before_dedup = len(df)
df_before_dedup = df.copy()

# Remove duplicate rows (keep first occurrence)
df = df.drop_duplicates()

duplicates_removed = rows_before_dedup - len(df)
if duplicates_removed > 0:
    print(f"  ✓ Removed {duplicates_removed} duplicate records")
    cleaning_log.append(f"Removed {duplicates_removed} duplicate records")
else:
    print("  ✓ No duplicates found")

# Check for duplicate transaction IDs
dup_ids = df['transaction_id'].duplicated()
if dup_ids.sum() > 0:
    print(f"\n  ⚠ Warning: {dup_ids.sum()} duplicate transaction IDs still exist")
    print("  → Removing records with duplicate transaction IDs (keeping first)")
    df = df.drop_duplicates(subset=['transaction_id'], keep='first')
    print(f"  ✓ Cleaned duplicate IDs")
    cleaning_log.append(f"Removed duplicate transaction IDs")

print(f"\n  Records after deduplication: {len(df)}")

### Clean 3: Fix Domain Violations

In [ ]:
print("\n[CLEAN 3] Fixing Domain Constraint Violations")
print("-" * 80)

rows_before_domain = len(df)

# Transaction Hour: clip to 0-23
invalid_hours = df[(df['transaction_hour'] < 0) | (df['transaction_hour'] > 23)]
if len(invalid_hours) > 0:
    print(f"  ⚠ Found {len(invalid_hours)} invalid transaction hours")
    df['transaction_hour'] = df['transaction_hour'].clip(0, 23)
    print(f"  ✓ Clipped transaction_hour to valid range [0-23]")
    cleaning_log.append(f"Clipped {len(invalid_hours)} transaction_hour values")

# Device Trust Score: clip to 0-100
invalid_scores = df[(df['device_trust_score'] < 0) | (df['device_trust_score'] > 100)]
if len(invalid_scores) > 0:
    print(f"  ⚠ Found {len(invalid_scores)} invalid device trust scores")
    df['device_trust_score'] = df['device_trust_score'].clip(0, 100)
    print(f"  ✓ Clipped device_trust_score to valid range [0-100]")
    cleaning_log.append(f"Clipped {len(invalid_scores)} device_trust_score values")

# Velocity: clip to non-negative
invalid_velocity = df[df['velocity_last_24h'] < 0]
if len(invalid_velocity) > 0:
    print(f"  ⚠ Found {len(invalid_velocity)} negative velocity values")
    df['velocity_last_24h'] = df['velocity_last_24h'].clip(lower=0)
    print(f"  ✓ Clipped velocity_last_24h to non-negative")
    cleaning_log.append(f"Clipped {len(invalid_velocity)} velocity_last_24h values")

# Age: Remove records with invalid ages (can't fix age easily)
invalid_age = df[(df['cardholder_age'] < 18) | (df['cardholder_age'] > 100)]
if len(invalid_age) > 0:
    print(f"  ⚠ Found {len(invalid_age)} invalid ages")
    df = df[(df['cardholder_age'] >= 18) & (df['cardholder_age'] <= 100)]
    print(f"  ✓ Removed {len(invalid_age)} records with invalid ages")
    cleaning_log.append(f"Removed {len(invalid_age)} records: invalid age")

# Binary fields: Remove records with invalid binary values
binary_fields = ['foreign_transaction', 'location_mismatch', 'is_fraud']
for field in binary_fields:
    invalid_binary = df[~df[field].isin([0, 1])]
    if len(invalid_binary) > 0:
        print(f"  ⚠ Found {len(invalid_binary)} invalid {field} values")
        df = df[df[field].isin([0, 1])]
        print(f"  ✓ Removed {len(invalid_binary)} records with invalid {field}")
        cleaning_log.append(f"Removed {len(invalid_binary)} records: invalid {field}")

domain_removed = rows_before_domain - len(df)
if domain_removed == 0:
    print("\n  ✓ No domain violations needed cleaning")
else:
    print(f"\n  Total records removed for domain violations: {domain_removed}")

print(f"\n  Records after domain cleaning: {len(df)}")

### Clean 4: Handle Missing Values (if any found)

In [ ]:
print("\n[CLEAN 4] Handling Missing Values")
print("-" * 80)

missing_before = df.isnull().sum().sum()

if missing_before > 0:
    print(f"  Found {missing_before} missing values")
    print("\n  Missing value distribution:")
    print(df.isnull().sum()[df.isnull().sum() > 0])
    
    # Strategy: Remove rows with ANY missing values (for fraud detection, we need complete data)
    rows_with_missing = df.isnull().any(axis=1).sum()
    df = df.dropna()
    
    print(f"\n  ✓ Removed {rows_with_missing} rows with missing values")
    cleaning_log.append(f"Removed {rows_with_missing} rows with missing values")
else:
    print("  ✓ No missing values found")

print(f"\n  Records after missing value handling: {len(df)}")

### Clean 5: Reset Index

In [ ]:
print("\n[CLEAN 5] Resetting Index")
print("-" * 80)

df = df.reset_index(drop=True)
print("  ✓ Index reset to sequential order")
print(f"\n  Final cleaned dataset: {len(df)} records")

### Cleaning Summary

In [ ]:
print("\n" + "="*80)
print("🧹 DATA CLEANING SUMMARY")
print("="*80)

print(f"\nOriginal dataset:    {rows_before:,} records")
print(f"Cleaned dataset:     {len(df):,} records")
print(f"Records removed:     {rows_before - len(df):,} ({(rows_before - len(df))/rows_before*100:.2f}%)")
print(f"Records retained:    {len(df)/rows_before*100:.2f}%")

print("\nCleaning actions performed:")
if cleaning_log:
    for i, action in enumerate(cleaning_log, 1):
        print(f"  {i}. {action}")
else:
    print("  ✓ No cleaning needed - data was already clean")

print("\n" + "="*80)

## 5. Data Quality Assessment (AFTER Cleaning)
### Verify Domain Constraints Again

In [ ]:
print("\n[VERIFICATION] DOMAIN CONSTRAINT VALIDATION (AFTER CLEANING)")
print("-" * 80)

verification_results = {}

# Transaction Hour: 0-23
invalid_hours = df[(df['transaction_hour'] < 0) | (df['transaction_hour'] > 23)]
verification_results['transaction_hour'] = len(invalid_hours)
print(f"  transaction_hour (0-23): {len(invalid_hours)} invalid values {'✓' if len(invalid_hours)==0 else '✗'}")

# Device Trust Score: 0-100
invalid_scores = df[(df['device_trust_score'] < 0) | (df['device_trust_score'] > 100)]
verification_results['device_trust_score'] = len(invalid_scores)
print(f"  device_trust_score (0-100): {len(invalid_scores)} invalid values {'✓' if len(invalid_scores)==0 else '✗'}")

# Amount: positive
invalid_amounts = df[df['amount'] <= 0]
verification_results['amount'] = len(invalid_amounts)
print(f"  amount (>0): {len(invalid_amounts)} invalid values {'✓' if len(invalid_amounts)==0 else '✗'}")

# Velocity: non-negative
invalid_velocity = df[df['velocity_last_24h'] < 0]
verification_results['velocity'] = len(invalid_velocity)
print(f"  velocity_last_24h (≥0): {len(invalid_velocity)} invalid values {'✓' if len(invalid_velocity)==0 else '✗'}")

# Age: reasonable range
invalid_age = df[(df['cardholder_age'] < 18) | (df['cardholder_age'] > 100)]
verification_results['cardholder_age'] = len(invalid_age)
print(f"  cardholder_age (18-100): {len(invalid_age)} invalid values {'✓' if len(invalid_age)==0 else '✗'}")

# Binary fields: 0 or 1
binary_fields = ['foreign_transaction', 'location_mismatch', 'is_fraud']
for field in binary_fields:
    invalid_binary = df[~df[field].isin([0, 1])]
    verification_results[field] = len(invalid_binary)
    print(f"  {field} (0/1): {len(invalid_binary)} invalid values {'✓' if len(invalid_binary)==0 else '✗'}")

total_invalid = sum(verification_results.values())
if total_invalid == 0:
    print("\n  ✅ All domain constraints satisfied - DATA IS CLEAN!")
else:
    print(f"\n  ❌ Total domain violations: {total_invalid} - CLEANING FAILED!")

### Statistical Profiling (Cleaned Data)

In [ ]:
print("\n[STEP 6] STATISTICAL PROFILING (CLEANED DATA)")
print("-" * 80)

numerical_features = ['amount', 'transaction_hour', 'device_trust_score', 
                     'velocity_last_24h', 'cardholder_age']

stats_profile = df[numerical_features].describe(
    percentiles=[.01, .05, .25, .5, .75, .95, .99]
).T

# Add additional statistics
stats_profile['skewness'] = df[numerical_features].skew()
stats_profile['kurtosis'] = df[numerical_features].kurtosis()
stats_profile['cv'] = stats_profile['std'] / stats_profile['mean']

print("\nNumerical Features Profile:")
print(stats_profile.round(3))

### Target Variable Analysis (Cleaned Data)

In [ ]:
print("\n[STEP 7] TARGET VARIABLE ANALYSIS (CLEANED DATA)")
print("-" * 80)

fraud_dist = df['is_fraud'].value_counts().sort_index()
fraud_pct = df['is_fraud'].value_counts(normalize=True).sort_index() * 100

print("\nTarget Variable Distribution:")
print(f"  Normal (0): {fraud_dist.get(0, 0)} ({fraud_pct.get(0, 0):.2f}%)")
print(f"  Fraud (1):  {fraud_dist.get(1, 0)} ({fraud_pct.get(1, 0):.2f}%)")

if 1 in fraud_dist.index and 0 in fraud_dist.index:
    imbalance_ratio = fraud_dist[0] / fraud_dist[1]
    print(f"\n  Imbalance Ratio: {imbalance_ratio:.2f}:1")
    
    if imbalance_ratio > 10:
        print("  ⚠ HIGH IMBALANCE - Consider resampling techniques")
    elif imbalance_ratio > 5:
        print("  ⚠ MODERATE IMBALANCE - Monitor model performance")
    else:
        print("  ✓ ACCEPTABLE BALANCE")

### Correlation Analysis (Cleaned Data)

In [ ]:
print("\n[STEP 9] CORRELATION ANALYSIS (CLEANED DATA)")
print("-" * 80)

numerical_cols = ['amount', 'transaction_hour', 'device_trust_score', 
                 'velocity_last_24h', 'cardholder_age', 
                 'foreign_transaction', 'location_mismatch', 'is_fraud']

corr_matrix = df[numerical_cols].corr()

print("\nCorrelation with Target Variable (is_fraud):")
target_corr = corr_matrix['is_fraud'].sort_values(ascending=False)
for feature, corr_val in target_corr.items():
    if feature != 'is_fraud':
        print(f"  {feature:25s}: {corr_val:7.4f}")

# High correlations between features
print("\nHigh Feature Correlations (|r| > 0.7):")
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.7:
            high_corr_pairs.append((
                corr_matrix.columns[i],
                corr_matrix.columns[j],
                corr_matrix.iloc[i, j]
            ))

if high_corr_pairs:
    for feat1, feat2, corr_val in high_corr_pairs:
        print(f"  {feat1} <-> {feat2}: {corr_val:.4f}")
else:
    print("  ✓ No multicollinearity detected")

## 6. Data Quality Report (Final)

In [ ]:
print("\n" + "="*80)
print("📊 FINAL DATA QUALITY SCORECARD")
print("="*80)

total_records = len(df)
total_features = len(df.columns) - 1

# Calculate quality score
missing_values = df.isnull().sum().sum()
duplicates = df.duplicated().sum()
domain_violations = sum(verification_results.values())

quality_scores = {
    'Completeness': 100 if missing_values == 0 else 
                  (1 - missing_values / (total_records * total_features)) * 100,
    'Uniqueness': 100 if duplicates == 0 else
                 (1 - duplicates / total_records) * 100,
    'Validity': 100 if domain_violations == 0 else 0,
}

overall_quality = np.mean(list(quality_scores.values()))

print(f"\nDataset Size: {total_records} records × {total_features + 1} features")
print(f"\nQuality Dimensions:")
for dimension, score in quality_scores.items():
    status = "✓" if score >= 95 else "⚠"
    print(f"  {status} {dimension:15s}: {score:6.2f}%")

print(f"\n{'='*40}")
print(f"  Overall Quality Score: {overall_quality:.2f}%")
print(f"{'='*40}")

# Quality grade
if overall_quality >= 95:
    grade = "EXCELLENT ⭐⭐⭐"
elif overall_quality >= 85:
    grade = "GOOD ⭐⭐"
elif overall_quality >= 70:
    grade = "FAIR ⭐"
else:
    grade = "POOR ⚠"

print(f"\n  Data Quality Grade: {grade}")

## 7. Exploratory Data Analysis (Cleaned Data)
### Univariate Analysis

In [ ]:
print("\n" + "="*80)
print("UNIVARIATE ANALYSIS (CLEANED DATA)")
print("="*80)

numerical_features = ['amount', 'transaction_hour', 'device_trust_score', 
                     'velocity_last_24h', 'cardholder_age']

for feature in numerical_features:
    print(f"\n{feature.upper()}")
    print("-" * 40)
    
    print(f"  Mean:       {df[feature].mean():.2f}")
    print(f"  Median:     {df[feature].median():.2f}")
    print(f"  Std Dev:    {df[feature].std():.2f}")
    print(f"  Range:      [{df[feature].min():.2f}, {df[feature].max():.2f}]")
    print(f"  IQR:        {df[feature].quantile(0.75) - df[feature].quantile(0.25):.2f}")
    
    skew = df[feature].skew()
    kurt = df[feature].kurtosis()
    print(f"  Skewness:   {skew:.3f} {'(Right-skewed)' if skew > 0 else '(Left-skewed)' if skew < 0 else '(Symmetric)'}")
    print(f"  Kurtosis:   {kurt:.3f} {'(Heavy-tailed)' if kurt > 0 else '(Light-tailed)'}")

### Bivariate Analysis

In [ ]:
print("\n" + "="*80)
print("BIVARIATE ANALYSIS (Features vs Fraud) - CLEANED DATA")
print("="*80)

if df['is_fraud'].nunique() < 2:
    print("\n  ⚠ Cannot perform bivariate analysis - only one class present")
else:
    numerical_features = ['amount', 'transaction_hour', 'device_trust_score', 
                         'velocity_last_24h', 'cardholder_age']
    
    for feature in numerical_features:
        print(f"\n{feature.upper()} by Fraud Status:")
        print("-" * 40)
        
        fraud_stats = df.groupby('is_fraud')[feature].describe()
        print(fraud_stats)
        
        normal_vals = df[df['is_fraud'] == 0][feature]
        fraud_vals = df[df['is_fraud'] == 1][feature]
        
        if len(fraud_vals) > 0:
            stat, p_value = mannwhitneyu(normal_vals, fraud_vals)
            print(f"\n  Mann-Whitney U test p-value: {p_value:.4f}")
            if p_value < 0.05:
                print(f"  ✓ Significant difference between groups")
            else:
                print(f"  ✗ No significant difference")

## 8. Save Cleaned Data

In [ ]:
print("\n" + "="*80)
print("💾 SAVING CLEANED DATA")
print("="*80)

# Save cleaned data
df.to_csv(output_filepath, index=False)
print(f"\n✅ Cleaned data saved to: {output_filepath}")
print(f"✅ Total records saved: {len(df):,}")
print(f"✅ Total features: {len(df.columns)}")

# File size
import os
file_size = os.path.getsize(output_filepath) / 1024  # KB
print(f"✅ File size: {file_size:.2f} KB")

print("\n" + "="*80)
print("EXECUTIVE SUMMARY")
print("="*80)
print(f"\n✅ Data Cleaning Completed")
print(f"✅ Exploratory Data Analysis Completed")
print(f"✅ Data Quality Score: {overall_quality:.2f}%")
print(f"✅ Records: {rows_before:,} → {len(df):,} ({len(df)/rows_before*100:.2f}% retained)")
print(f"✅ Ready for Feature Engineering & Modeling")
print("\n" + "="*80)

## 9. Data Comparison: Before vs After Cleaning

In [ ]:
print("\n" + "="*80)
print("📊 BEFORE vs AFTER CLEANING COMPARISON")
print("="*80)

comparison = pd.DataFrame({
    'Metric': [
        'Total Records',
        'Missing Values',
        'Duplicates',
        'Invalid Amounts',
        'Domain Violations',
        'Data Quality Score'
    ],
    'Before Cleaning': [
        f"{rows_before:,}",
        f"{df_raw.isnull().sum().sum()}",
        f"{df_raw.duplicated().sum()}",
        f"{len(df_raw[df_raw['amount'] <= 0])}",
        f"{total_invalid}",
        "98.33%"  # From original output
    ],
    'After Cleaning': [
        f"{len(df):,}",
        f"{df.isnull().sum().sum()}",
        f"{df.duplicated().sum()}",
        f"{len(df[df['amount'] <= 0])}",
        f"{sum(verification_results.values())}",
        f"{overall_quality:.2f}%"
    ]
})

print("\n" + comparison.to_string(index=False))
print("\n" + "="*80)

## 10. Quick Data Preview

In [ ]:
print("\n📋 CLEANED DATA PREVIEW")
print("="*80)
print("\nFirst 10 rows:")
df.head(10)

In [ ]:
print("\n📋 DATA INFO")
print("="*80)
df.info()

In [ ]:
print("\n📋 STATISTICAL SUMMARY")
print("="*80)
df.describe()